In [24]:
import os
import requests
import json


In [26]:
# Load API key from environment variable (you can also directly assign it here, but not recommended)
weather_api_key = os.getenv('fe7e7af7af3f4b5a94f110620241710')
stock_api_key = os.getenv('cs8f0jpr01qsceftno70cs8f0jpr01qsceftno7g')

In [27]:

# Define real API calls for current weather
def get_current_weather(location):
    base_url = "http://api.weatherapi.com/v1/current.json"
    params = {
        "key": "fe7e7af7af3f4b5a94f110620241710",  # Use your API key
        "q": location
    }

    response = requests.get(base_url, params=params)
    "Status codes let you know if your request was successfully processed (e.g., 200 OK) or if something went wrong (e.g., 404 Not Found)."

    if response.status_code == 200:
        data = response.json()
        return {
            "weather": data['current']['condition']['text'],
            "temperature": data['current']['temp_f']
        }
    else:
        return {"error": "Unable to fetch weather data"}

In [28]:
# Define real API call for stock price
def get_stock_price(ticker):
    base_url = f"https://finnhub.io/api/v1/quote"
    params = {
        "symbol": ticker,
        "token": "cs8f0jpr01qsceftno70cs8f0jpr01qsceftno7g"  # Use your API key
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        return {
            "price": data.get('c', "unknown")  # 'c' is the current price in Finnhub API
        }
    else:
        return {"error": "Unable to fetch stock price"}

In [29]:
def get_vegetable_wholesaleprice(vegetable_name):
    # Simulate data for vegetable price price
    vegetable_data = {
        "Onion": {"price": 46.0},
        "Tomato": {"price": 49.0},
        "Potato": {"price": 38.0},
        "Beetroot": {"price": 33.0},
        "Carrot": {"price": 40},
        "Cabbage": {"price": 25},
        "Brinjal": {"price": 35},
        "lemon": {"price" :71},
        "Radish": {"price":30},
        "Spinach": {"price":12}
    }
    return vegetable_data.get(vegetable_name, {"price": "unknown"})


In [30]:
# Define the function metadata
functions_metadata = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather for a specified location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state or city and country for which you want the weather."
                    }
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_stock_price",
            "description": "Get the current stock price for a specified ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "The stock ticker symbol, e.g., 'AAPL' for Apple"
                    }
                },
                "required": ["ticker"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_vegetable_wholesaleprice",
            "description": "Get the vegetable wholesale price for a specified vegetable ",
            "parameters": {
                "type": "object",
                "properties": {
                    "vegetable_name": {
                        "type": "string",
                        "description": "The name of the vegetable for which you want the wholesale price"
                    }
                },
                "required": ["vegetable_name"]
            }
        }
    }
]

In [31]:
class Gemini_15_LLM:
    def __init__(self, functions_metadata):
        self.functions = {
            "get_current_weather": get_current_weather,
            "get_stock_price": get_stock_price,
            "get_vegetable_wholesaleprice": get_vegetable_wholesaleprice  # Corrected function name
        }
        self.functions_metadata = functions_metadata

    def call_function(self, function_name, **args):
        if function_name in self.functions:
            return self.functions[function_name](**args)
        else:
            return {"error": "Function not found"}

    def respond_to_prompt(self, prompt):
        if "weather" in prompt.lower() and "stock" in prompt.lower() and "vegetable" in prompt.lower():
            return {
                "functions": [
                    {"name": "get_current_weather", "arguments": {"location": "Kuppam,KPM"}},
                    {"name": "get_stock_price", "arguments": {"ticker": "AAPL"}},
                    {"name": "get_vegetable_wholesaleprice", "arguments": {"vegetable_name": "Carrot"}}
                ]
            }
        elif "weather" in prompt.lower():
            return {"name": "get_current_weather", "arguments": {"location": "Kuppam,KPM"}}
        elif "stock" in prompt.lower():
            return {"name": "get_stock_price", "arguments": {"ticker": "AAPL"}}
        elif "vegetable" in prompt.lower():
            return {"name": "get_vegetable_wholesaleprice", "arguments": {"vegetable_name": "Carrot"}}
        else:
            return "No function call required"

In [32]:

# Create an instance of the Gemini 15-like LLM
gemini_15_llm = Gemini_15_LLM(functions_metadata)

In [33]:
# System prompt (emulates Gemini system awareness)
system_prompt = f"""You are an advanced assistant with access to the following functions:
{json.dumps(functions_metadata, indent=4)}

To use these functions respond with:
{{"name":"function_name","arguments":{{"arg_1":"value_1","arg_2":"value_2",...}}}}

You can also chain multiple function calls when necessary."""

In [34]:
# Simulate user queries
user_queries = [

    "What is the current weather in Kuppam,KPM?"
]

In [35]:

# Process each query and store in JSON format
responses = []

In [36]:
for query in user_queries:
    result = {}
    result["User"] = query
    response = gemini_15_llm.respond_to_prompt(query)

    if isinstance(response, dict) and "functions" in response:
        # If multiple function calls are suggested
        result["Assistant"] = []
        for func in response["functions"]:
            function_name = func['name']
            function_args = func.get('arguments', {})  # Safely get 'arguments', defaulting to an empty dict
            function_result = gemini_15_llm.call_function(function_name, **function_args)

            result["Assistant"].append({
                "Function Name": function_name,
                "Arguments": function_args,
                "Result": function_result
            })
    elif isinstance(response, dict) and "name" in response:
        function_name = response['name']
        function_args = response.get('arguments', {})  # Safely get 'arguments', defaulting to an empty dict
        function_result = gemini_15_llm.call_function(function_name, **function_args)

        result["Assistant"] = {
            "Function Name": function_name,
            "Arguments": function_args,
            "Result": function_result
        }
    else:
        result["Assistant"] = {"Message": response}

    responses.append(result)


In [37]:
print(responses)

[{'User': 'What is the current weather in Kuppam,KPM?', 'Assistant': {'Function Name': 'get_current_weather', 'Arguments': {'location': 'Kuppam,KPM'}, 'Result': {'weather': 'Mist', 'temperature': 90.1}}}]


In [38]:
# Convert the list of responses to JSON format
json_output = json.dumps(responses, indent=4)


# Print the JSON output
print(json_output)

[
    {
        "User": "What is the current weather in Kuppam,KPM?",
        "Assistant": {
            "Function Name": "get_current_weather",
            "Arguments": {
                "location": "Kuppam,KPM"
            },
            "Result": {
                "weather": "Mist",
                "temperature": 90.1
            }
        }
    }
]
